**Learning outcomes**

* Derive the Trotter-Suzuki decomposition.
* Use Trotterization to simulate a simple physical system.

In [2]:
import numpy as np
import pennylane as qml

Let's now consider two electrons in a magnetic field. If they are far away from each other, then the total Hamiltonian (energy) will just be a sum of Hamiltonians:
![equation](./images/H.5.1.1.png)
where $Z_0 = Z\otimes I$ is the Pauli Z acting on the first electron, and $Z_1 = I \otimes Z$ acts on the second. Since the electrons are independent, the unitary which evolves this two-electron system in time is two copies of the unitary $e^{i\alpha tZ}$, $\alpha = eB/2m_e$. In circuit form:
![equation](./images/H.5.1.2.png)

**Codercise H.5.1.**
Complete the function below for simulating two distant electrons in a magnetic field.

In [3]:
n_bits=2
dev = qml.device("default.qubit", wires=range(n_bits))

@qml.qnode(dev)
def two_distant_spins(B, time):
    """Circuit for evolving the state of two distant electrons in a magnetic field.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        time (float): The time we evolve the electron wavefunction for.

    Returns:
        array[complex]: The quantum state after evolution.
    """
    e = 1.6e-19
    m_e = 9.1e-31
    alpha = B*e/(2*m_e)

    qml.RZ(phi=-2 * alpha * time, wires=0)
    qml.RZ(phi=-2 * alpha * time, wires=1)

    return qml.state()

Another way to express this circuit is in matrix form,$U(t) = e^{i\alpha tZ_0}e^{i\alpha tZ_1}$ . Schrödinger's equation tells us, on the other hand, that $U(t) = e^{-i\hat{H}/\hbar} = e^{i \alpha t(Z_0 Z_1)}$. These two expressions appear to be equal by virtue of the rule $e^{x+y} = e^xe^y$. But not so fast! We are dealing with matrices rather than numbers, and this makes a huge change.
It turns out that, for matrices $A$ and $B$,$e^{A+B} = e^Ae^B$ is only tru when $A$ and $B$ can be freely reordered (commute), or $AB = BA$. We say such metrices **commute**. When term do not commute, exponentiating their sum is hard. As a concrete example, consider two nearby electrons, who can feel each other's magnetic field. The Hamiltonian will now include an interaction term since the spins of these electrons would like to anti-align:
![picture](./images/H.5.2.1.png)
For simplicity, we'll make this interaction term proportional to $X_0X_1$:
![equation](./images/H.5.2.2.png)
where $J$ depends on how close they are. (The factor of $(\hbar / 2)^2$) comes from converting spins into Pauli operators). To time evolve this system, we need to exponentiate these non-commuting terms, which is hard.

Thankfully, there is a beautiful approximation called the **Trotter-Suzuki decomposition**:
$$e^{A+B} = \lim_{n\to\infty}(e^{A/n}e^{B/n})^n$$
This lead to a simple algorithm called **Trotterization** for simulating a quantum system.